In [ ]:
import pandas as pd
import numpy as np
import torch
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
from utils.neural_nets import NN_Module_2C
from utils.params import fig_dir
from utils.metrics import mean_relative_errors
from utils.charts import plot_freq_range_errors
from tqdm import tqdm

%config InlineBackend.figure_format ='retina'
mpl.style.use("ggplot")
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
df = pd.read_csv("data/flow_data_2c_1.csv").astype(np.float32)
df = df.sample(200_000)
df.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    df.iloc[:, -140:], df.iloc[:, :5], test_size=0.1, random_state=42
)

print(X_train.shape, Y_train.shape)

In [ ]:
np.save("data/predictions/freq_ranges/bimodal/X_train", X_train, allow_pickle=False)
np.save("data/predictions/freq_ranges/bimodal/X_test", X_test, allow_pickle=False)
np.save("data/predictions/freq_ranges/bimodal/Y_train", Y_train, allow_pickle=False)
np.save("data/predictions/freq_ranges/bimodal/Y_test", Y_test, allow_pickle=False)

In [ ]:
freq_ranges = np.load("data/predictions/freq_ranges/freq_ranges.npy", allow_pickle=True)

In [ ]:
with tqdm(total=28) as pbar:
    for row in freq_ranges:
        for freq_range in row:
            storage_modulus_indices = list(
                range(min(freq_range), max(freq_range))
            )
            loss_modulus_indices = list(
                range(min(freq_range) + 70, max(freq_range) + 70)
            )
            X_train_freq_range = X_train[
                :, storage_modulus_indices + loss_modulus_indices
            ]
            X_test_freq_range = X_test[
                :, storage_modulus_indices + loss_modulus_indices
            ]

            net = NeuralNetRegressor(
                module=NN_Module_2C,
                module__num_targets=5,
                module__num_features=X_train_freq_range.shape[1],
                criterion=torch.nn.MSELoss,
                optimizer=torch.optim.AdamW,
                lr=2e-05,
                max_epochs=100,
                verbose=0,
                device="cuda" if torch.cuda.is_available() else "cpu",
            )
            tt = TransformedTargetRegressor(
                regressor=net, transformer=MinMaxScaler()
            )
            pipe = Pipeline([("scale", MinMaxScaler()), ("tt", tt)])

            pipe.fit(X_train_freq_range, Y_train)

            Y_pred = pipe.predict(X_test_freq_range)
            np.save(
                f"data/predictions/freq_ranges/bimodal/Y_pred_{pbar.n}",
                Y_pred,
                allow_pickle=False,
            )

            pbar.update(1)

In [ ]:
errors = []
for i in range(28):
    Y_pred = np.load(
        f"data/predictions/freq_ranges/bimodal/Y_pred_{i}.npy", allow_pickle=False
    )
    rel_errs = mean_relative_errors(Y_test, Y_pred)
    errors.append(np.mean(rel_errs))
errors = np.array(errors)

In [ ]:
plot_freq_range_errors(errors, unimodal=False)

plt.savefig(
    fig_dir + "/freq_ranges_2c.png", dpi=300, bbox_inches="tight", pad_inches=0
)